# Building a Multi-Output Model to Predict Macronutrient Profile

In this notebook we are going to improve on the current model to predict calores, and instead we will predict all 3 macronutrients. Here is how it will work:

- The user will input a recipe name.
- Then a list of suggested ingredients will appear, and the user can optionally edit this by either adding/subtracting ingredients or modifying quantities
    - This will be obtained for now by just running the cosine similarity with the recipe names in the data, and taking the ingredients from the top 5 or so results.
    - Quantities may be ignored if model can't handle numeric inputs, but LLMs can if going that route.
    - Being able to edit this could be a paid feature, i.e. changing quantities or adding/subtracting ingredients.
- The model will then take as input, the recipe name concatenated with the ingredients list. Perhaps with quantities as well if using an LLM.
    - Consider adding dietery type/preference here as well.
- Finally, a multi-output regression model will be used to predict the macronutrients, and we can take ratios by dividing each macro by the total calorie count. The aim here is that during training, the model will learn to optimize its parameters to minimize the error between the predicted and ground truth ratios derived from the normalized macronutrient values.

In [10]:
#keeping all imports at the top
import pandas as pd
import ast

for module_name in ['pandas',]:
    module = __import__(module_name)
    print(f"{module_name}: {module.__version__}")

pandas: 2.2.0


## EDA

In [5]:
df = pd.read_csv('../recipes.csv')
df.columns

Index(['uri', 'label', 'image', 'source', 'url', 'shareAs', 'yield',
       'dietLabels', 'healthLabels', 'cautions', 'ingredientLines',
       'ingredients', 'calories', 'totalWeight', 'totalTime', 'cuisineType',
       'mealType', 'dishType', 'totalNutrients', 'totalDaily', 'digest',
       'tags'],
      dtype='object')

In [6]:
ingredients_col = df['ingredients'].apply(ast.literal_eval)